In [146]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import plotly as ply
import os
import re
import requests
import json
from IPython.core.display import HTML

# Load Shot Data

In [147]:
shot_data = pd.read_csv('data/fantasy-league/shot_data.csv', index_col=0)
shot_data['Timestamp'] = pd.to_datetime(shot_data['Timestamp'], format='%Y-%m-%d') 

In [148]:
shot_data.head()

,Timestamp,Score,Player Advantage,Minute,Player,Squad,Against,Outcome,Distance,Body Part,Notes,SCA 1 Player,SCA 1 Event,SCA 2 Player,SCA 2 Event,Threat
0,2016-08-13 12:30:00,0.0,0.0,147,Riyad Mahrez,Leicester City,Hull City,Goal,NaN,NaN,Penalty Kick,Goal,NaN,NaN,NaN,720.0
1,2016-08-13 17:30:00,1.0,0.0,4,Sergio Agüero,Manchester City,Sunderland,Goal,NaN,NaN,Penalty Kick,—,NaN,NaN,NaN,720.0
2,2016-08-15 20:00:00,1.0,0.0,147,Eden Hazard,Chelsea,West Ham United,Goal,NaN,NaN,Penalty Kick,Yellow Card,NaN,NaN,NaN,627.0
3,2016-08-19 20:00:00,2.0,0.0,152,Zlatan Ibrahimović,Manchester United,Southampton,Goal,NaN,NaN,Penalty Kick,Goal,NaN,NaN,NaN,627.0
4,2016-08-20 12:30:00,1.0,0.0,27,Sergio Agüero,Manchester City,Stoke City,Goal,NaN,NaN,Penalty Kick,Yellow Card,NaN,NaN,NaN,NaN


# Load Non-shot Data

In [149]:
non_shot_data = pd.read_csv('data/non-shot-xG/non_shot_data.csv', index_col=0)
non_shot_data['Date'] = pd.to_datetime(non_shot_data['Date'], format='%d/%m/%Y')

In [150]:
non_shot_data.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HF,AF,HC,AC,HY,AY,HR,AR
GameID,,,,,,,,,,,,,,,,,,
1,2008-08-16,Arsenal,West Brom,1,0,H,1,0,H,H Webb,11,8,7,5,0,0,0,0
2,2008-08-16,Bolton,Stoke,3,1,H,3,0,H,C Foy,13,12,4,3,1,2,0,0
3,2008-08-16,Everton,Blackburn,2,3,A,1,1,D,A Marriner,11,9,3,5,2,2,0,0
4,2008-08-16,Hull,Fulham,2,1,H,1,1,D,P Walton,10,9,5,6,3,0,0,0
5,2008-08-16,Middlesbrough,Tottenham,2,1,H,0,0,D,M Atkinson,11,12,7,9,1,2,0,0


# Match Data

In [151]:
non_shot_data['Date'] = [d.date() for d in non_shot_data['Date']]

In [152]:
arr = np.asarray([d for d in non_shot_data['Date']]) == pd.to_datetime('2018-01-12').date()

In [153]:
non_shot_data_groups = non_shot_data.groupby('Date').groups

In [154]:
shot_data['MatchID'] = np.NaN

## Add MatchID

In [155]:
errors = []
try:
    for i, s in shot_data.iterrows():
        date = s['Timestamp'].date()
        if date in non_shot_data_groups:
            indices = non_shot_data_groups[date]
            group = non_shot_data.iloc[indices-1]
            home = group['HomeTeam'] == s['Squad']
            if sum(home) > 0:
                shot_data.loc[i, 'MatchID'] = group.loc[home].index[0]
                continue
            away = group['AwayTeam'] == s['Squad']
            if sum(away) > 0:
                shot_data.loc[i, 'MatchID'] = group.loc[away].index[0]
                continue

        else:
            errors.append(i)
except:
    print(i)

In [156]:
shot_data = shot_data.loc[shot_data['Timestamp'] < pd.to_datetime('2021-01-04', format='%Y-%m-%d')]

In [143]:
path = os.path.join(os.getcwd(), "output/shot_data.csv")
shot_data.to_csv(path, index=False)